# Introduction to Data Science - miniproject
## Preprocessing of data

## Helsinki districts to a clean data frame and csv

In [22]:
import pandas as pd
import numpy as np

# Install required version of openpyxl if needed
#!pip install openpyxl==3.0.10

# Read excel file from repository, remove the introductory row 0 and the last row without real data
# Original web address to file: 'https://www.hel.fi/hel2/tietokeskus/data/helsinki/Helsingin_aluejaot_2016.xlsx'
districts_fetch = pd.read_excel('../Data_sourcefiles/Helsingin_aluejaot_2016.xlsx', 
                          sheet_name='Aluejako', skiprows=1, skipfooter=1)

districts_fetch

,KOKOTUN,KUNTA,SUUR,PERUS,OSA,PIEN,Nimi,Helsingin tilastoaluetunnus,Tilastokeskuksen aluetunnus
0,910000000,91,0,0,0,0,0,091 Helsinki,091 Helsinki
1,911000000,91,1,0,0,0,Eteläinen,091 1 Eteläinen suurpiiri,0911 1. Eteläinen suurpiiri
2,911101000,91,1,101,0,0,Vironniemi,091 101 Vironniemen peruspiiri,09111 101 Vironniemen peruspiiri
3,911101010,91,1,101,10,0,Kruununhaka,091 10 Kruununhaka,09111010 010 Kruununhaka
4,9111010101,91,1,101,10,1,NaN,1,091110101 0101
...,...,...,...,...,...,...,...,...,...
590,9188015800,91,8,801,580,0,NaN,0,091815800 5800
591,918801591,91,8,801,591,0,Landbo,091 591 Landbo,09181591 591 Landbo
592,9188015910,91,8,801,591,0,NaN,0,091815910 5910
593,918801592,91,8,801,592,0,Puroniitty,091 592 Puroniitty,09181592 592 Puroniitty


In [51]:
districts = districts_fetch.copy()
# Remove unneeded columns
districts = districts.drop(['KOKOTUN', 'PIEN'], axis=1)

# Change numerical columns to ints 
districts = districts.astype({'KUNTA': int,
                        'SUUR': int,
                        'PERUS': int,
                        'OSA': int,
                       })

# Remove level for 'pienalue'
districts = districts[districts['Nimi'].notna()]

# Filter out the row for city level 
districts = districts[districts['SUUR'] != 0]

# Make a new column for the names of 'suurpiiri' and fill values down
districts['Suurpiiri'] = districts['Nimi'].where((districts['SUUR'] != 0) & (districts['PERUS'] == 0)).ffill()

# Make a new column for the names of 'peruspiiri' and fill values down
districts['Peruspiiri'] = districts['Nimi'].where((districts['PERUS'] != 0) & (districts['OSA'] == 0)).ffill()
# Replace name with 0 for level 'suurpiiri'
districts['Peruspiiri'] = districts['PERUS'].where(districts['PERUS'] == 0, districts['Peruspiiri'])

# Make a new column for the names of 'osa-alue' and fill values down
districts['Osa-alue'] = districts['Nimi'].where(districts['OSA'] != 0).ffill()
# Replace name with 0 for levels 'suurpiiri' and 'peruspiiri',
districts['Osa-alue'] = districts['OSA'].where(districts['OSA'] == 0, districts['Osa-alue'])

# Make a new column for names without å, ä, ö
districts['Helsingin tilastoaluetunnus 2'] = districts['Helsingin tilastoaluetunnus']
replacements = {'å':'a', 'Å':'A', 'ä':'a', 'Ä':'A', 'ö':'o', 'Ö':'O', }
for char in replacements.keys():
    districts['Helsingin tilastoaluetunnus 2'] = districts['Helsingin tilastoaluetunnus 2'].str.replace(char, replacements[char])

    
# Rename columns with English names
districts.columns = ['Municipality', 'MAJOR', 'BASIC', 'SUB', 'Name', 'Helsinki statistical area code', 
                     'Statistics Finland area code', 'Major district', 'District', 'Sub-district', 'Helsinki statistical area code 2']


districts.tail(20)

,Municipality,MAJOR,BASIC,SUB,Name,Helsinki statistical area code,Statistics Finland area code,Major district,District,Sub-district,Helsinki statistical area code 2
540,91,7,703,474,Kivikko,091 474 Kivikko,09173474 474 Kivikko,Itäinen,Mellunkylä,Kivikko,091 474 Kivikko
545,91,7,703,475,Kurkimäki,091 475 Kurkimäki,09173475 475 Kurkimäki,Itäinen,Mellunkylä,Kurkimäki,091 475 Kurkimaki
547,91,7,704,0,Vuosaari,091 704 Vuosaaren peruspiiri,09174 704 Vuosaaren peruspiiri,Itäinen,Vuosaari,0,091 704 Vuosaaren peruspiiri
548,91,7,704,541,Keski-Vuosaari,091 541 Keski-Vuosaari,09174541 541 Keski-Vuosaari,Itäinen,Vuosaari,Keski-Vuosaari,091 541 Keski-Vuosaari
555,91,7,704,542,Nordsjön kartano,091 542 Nordsjön kartano,09174542 542 Nordsjön kartano,Itäinen,Vuosaari,Nordsjön kartano,091 542 Nordsjon kartano
557,91,7,704,543,Uutela,091 543 Uutela,09174543 543 Uutela,Itäinen,Vuosaari,Uutela,091 543 Uutela
559,91,7,704,544,Meri-Rastila,091 544 Meri-Rastila,09174544 544 Meri-Rastila,Itäinen,Vuosaari,Meri-Rastila,091 544 Meri-Rastila
564,91,7,704,545,Kallahti,091 545 Kallahti,09174545 545 Kallahti,Itäinen,Vuosaari,Kallahti,091 545 Kallahti
568,91,7,704,546,Aurinkolahti,091 546 Aurinkolahti,09174546 546 Aurinkolahti,Itäinen,Vuosaari,Aurinkolahti,091 546 Aurinkolahti
572,91,7,704,547,Rastila,091 547 Rastila,09174547 547 Rastila,Itäinen,Vuosaari,Rastila,091 547 Rastila


In [52]:
# Write districts dataframe to csv file in data folder
districts.to_csv('../data/Helsinki_districts.csv', index=False)

## One-person households data

In [43]:
import pandas as pd
import numpy as np

# Install required version of openpyxl if needed
#!pip install openpyxl==3.0.10

# Read excel file from repository, remove the introductory rows 0-3 and the last row without real data
# Original web address to file: 'https://www.hel.fi/hel2/tietokeskus/data/helsinki/Vaestotilastot/2021/Hki_vaesto_taulu17c.xlsx'
# File downloadable at https://www.avoindata.fi/data/fi/dataset/yksinasuvat-yhden-hengen-asuntokunnat-helsingissukupuolen-ja-imukaan-peruspiireitt-2004-alkaen
'https://www.hel.fi/hel2/tietokeskus/data/helsinki/Helsingin_aluejaot_2016.xlsx'
ones = pd.read_excel('../Data_sourcefiles/Hki_vaesto_taulu17c_yksinasuvat.xlsx', 
                          sheet_name='2022', skiprows=4, skipfooter=1)

ones

,Sukupuoli,Yhteensä,-19,20-29,30-39,40-49,50-59,60-64,65-69,70-74,75-79,80-84,85-
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Koko Helsinki,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Yhteensä,175045.0,2266.0,36455.0,29909.0,19363.0,23806.0,12679.0,12200.0,12212.0,10130.0,7711.0,8314.0
3,Miehet,74602.0,774.0,16254.0,16608.0,11534.0,11258.0,4956.0,4094.0,3450.0,2511.0,1576.0,1587.0
4,Naiset,100443.0,1492.0,20201.0,13301.0,7829.0,12548.0,7723.0,8106.0,8762.0,7619.0,6135.0,6727.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,801 Östersundom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
212,Yhteensä,113.0,1.0,4.0,9.0,17.0,26.0,12.0,9.0,9.0,11.0,8.0,7.0
213,Miehet,68.0,1.0,2.0,7.0,11.0,16.0,7.0,6.0,5.0,7.0,3.0,3.0
214,Naiset,45.0,0.0,2.0,2.0,6.0,10.0,5.0,3.0,4.0,4.0,5.0,4.0


In [44]:
# Change column 'sukupuoli' to type string
ones = ones.astype({'Sukupuoli': str})

# Remove trailing spaces in column 'Sukupuoli'
ones['Sukupuoli'] = ones['Sukupuoli'].apply(lambda x: x.strip())


# Remove 'total' rows ('yhteensä')
ones = ones[ones['Sukupuoli'] != 'Yhteensä']

# Add new column for 'suurpiiri'
ones['Suurpiiri'] = ones['Sukupuoli'].where(ones['Sukupuoli'].str.contains('suurpiiri')).ffill()

# Add new column for 'peruspiiri'
ones['Peruspiiri'] = ones['Sukupuoli'].where((ones['Sukupuoli'] != 'Miehet') & (ones['Sukupuoli'] != 'Naiset')).ffill()

# Remove rows with empty values
ones = ones.dropna()

# Remove rows for level 'suurpiiri'
ones = ones[~ones['Peruspiiri'].str.contains('suurpiiri')]

# Change numerical columns to type int
ones = ones.astype({'Yhteensä': int,
                    -19: int,
                    '20-29': int,
                    '30-39': int, 
                    '50-59': int,
                    '60-64': int,
                    '65-69': int,
                    '70-74': int,
                    '75-79': int,
                    '80-84': int,
                    '85-': int})

# Translate 'sukupuoli' values to English
ones['Sukupuoli'] = ones['Sukupuoli'].replace({'Miehet': 'Men', 'Naiset': 'Women'})

# Rename columns with English names
ones.columns = ['Gender', 'All ages', 'Age -19', 'Age 20-29', 'Age 30-39', 'Age 40-49', 'Age 50-59', 'Age 60-64', 'Age 65-69', 'Age 70-74', 'Age 75-79', 'Age 80-84', 'Age 85', 'Major district', 'District']
# Add suffix to column names to point them to one-person households
ones = ones.add_prefix('One-person hh, ')

ones


,"One-person hh, Gender","One-person hh, All ages","One-person hh, Age -19","One-person hh, Age 20-29","One-person hh, Age 30-39","One-person hh, Age 40-49","One-person hh, Age 50-59","One-person hh, Age 60-64","One-person hh, Age 65-69","One-person hh, Age 70-74","One-person hh, Age 75-79","One-person hh, Age 80-84","One-person hh, Age 85","One-person hh, Major district","One-person hh, District"
13,Men,1306,18,293,248,210.0,195,80,67,76,53,43,23,1 Eteläinen suurpiiri,101 Vironniemi
14,Women,1925,31,428,230,161.0,213,106,159,174,168,135,120,1 Eteläinen suurpiiri,101 Vironniemi
18,Men,3117,28,847,647,517.0,427,191,132,109,84,70,65,1 Eteläinen suurpiiri,102 Ullanlinna
19,Women,4575,92,1254,759,379.0,481,278,298,314,310,210,200,1 Eteläinen suurpiiri,102 Ullanlinna
23,Men,5299,77,1598,1238,795.0,661,278,178,162,162,69,81,1 Eteläinen suurpiiri,103 Kampinmalmi
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,Women,5505,86,623,421,358.0,856,594,571,562,519,452,463,7 Itäinen suurpiiri,703 Mellunkylä
203,Men,3968,38,592,754,617.0,797,348,254,188,159,113,108,7 Itäinen suurpiiri,704 Vuosaari
204,Women,5606,49,556,439,426.0,986,544,557,617,518,467,447,7 Itäinen suurpiiri,704 Vuosaari
213,Men,68,1,2,7,11.0,16,7,6,5,7,3,3,8 Östersundomin suurpiiri,801 Östersundom


In [45]:
# Write one-person households dataframe to csv file in data folder
ones.to_csv('../data/Helsinki_oneperson_households.csv', index=False)

## Data for morbidity index

In [2]:
import pandas as pd
import numpy as np

# Install required version of openpyxl if needed
#!pip install openpyxl==3.0.10

In [3]:
# Latest published data is from 2021 but we will make an assumption that the indices are the same in 2022.
# Indices for the whole city are 100, districts are compared to that. 

# The morbidity index is calculated by means of three register variables describing the morbidity of the population:
# the share of people on disability pension among the working-age population, 
# the share of persons entitled to special reimbursements for medicines (representing long-term morbidity) 
# and the mortality rate of the population. The morbidity index is the average of these three variables.

# The chronic diseases index describes the prevalence of chronic diseases and conditions in the population 
# of the region compared to the average of the population of the entire city.
# The index includes the seven most common diseases entitling to reimbursable medical treatment: 
# asthma, diabetes, rheumatoid arthritis, psychoses, coronary artery disease, heart failure and hypertension. 
# The index is the average of these sub-indices.


# Read excel file from repository, remove the introductory rows 0-3 and the last row without real data
# Original web address to file: 'https://www.hel.fi/hel2/tietokeskus/data/helsinki/Terveys/Helsingin_sairastavuusindeksi_2021.xlsx'
# File downloadable at https://www.avoindata.fi/data/fi/dataset/helsingin-seudun-sairastavuusindeksi

morb_fetch = pd.read_excel('../Data_sourcefiles/Helsingin_sairastavuusindeksi_2021.xlsx', 
                          sheet_name='Sairastavuus-kansantindeksi', skiprows=1, skipfooter=10)

# Remove first 2 rows (empty and Helsinki-level) and second table on the right, keeping rows from index 1 and columns 0-6
morb_fetch = morb_fetch.iloc[2: , 0:7]

# Replace '7 ITÄINEN SUURIPIIRI' with '7 ITÄINEN SUURPIIRI' (remove extra I from 'SUURPIIRI')
morb_fetch['Alue'] = morb_fetch['Alue'].str.replace('7 ITÄINEN SUURIPIIRI', '7 ITÄINEN SUURPIIRI') 

# Change 'Väestö' to type int
morb_fetch = morb_fetch.astype({'Väestö 31.12.2021': int})

morb_fetch

,Alue,SAIRASTAVUUSINDEKSI,Kuolleisuusindeksi,Työkyvyttömyysindeksi,Lääkekorvausoikeusindeksi,KANSANTAUTI-INDEKSI,Väestö 31.12.2021
2,1 ETELÄINEN SUURPIIRI,75.814793,76.865467,61.854409,88.724502,79.527630,121157
3,Vironniemi,67.586539,69.787493,47.104700,85.867423,76.318461,12794
4,Ullanlinna,65.665101,73.060317,43.092436,80.842549,67.995023,24636
5,Kampinmalmi,82.818148,74.817956,78.870986,94.765503,88.586840,43738
6,Taka-Töölö,87.902482,95.685431,78.013245,90.008771,85.981219,15253
7,Lauttasaari,70.974337,72.153629,51.920147,88.849234,76.159888,24736
8,2 LÄNTINEN SUURPIIRI,99.731430,96.128562,103.524244,99.541484,99.705168,112045
9,Reijola,100.060517,87.922642,110.102059,102.156851,100.630765,16767
10,Munkkiniemi,85.070119,78.960570,82.743505,93.506284,90.378942,18844
11,Haaga,104.312980,111.773187,101.344437,99.821315,102.880702,27612


In [4]:
morb = morb_fetch.copy()

# Rename columns with English names
morb.columns = ['District', 'Morbidity index', 'Mortality index', 'Disability pension index', 
                'Medicine reimbursement index', 'Chronic diseases index', 'Population']

# Add new column for Major district ('suurpiiri')
morb['Major district'] = morb['District'].where(morb['District'].str.contains('SUURPIIRI')).ffill()

# Remove rows for level 'suurpiiri'
morb = morb[~morb['District'].str.contains('SUURPIIRI')]

# Add prefix to column names to point them to morbidity 
morb = morb.add_prefix('Morbidity, ')

morb

,"Morbidity, District","Morbidity, Morbidity index","Morbidity, Mortality index","Morbidity, Disability pension index","Morbidity, Medicine reimbursement index","Morbidity, Chronic diseases index","Morbidity, Population","Morbidity, Major district"
3,Vironniemi,67.586539,69.787493,47.104700,85.867423,76.318461,12794,1 ETELÄINEN SUURPIIRI
4,Ullanlinna,65.665101,73.060317,43.092436,80.842549,67.995023,24636,1 ETELÄINEN SUURPIIRI
5,Kampinmalmi,82.818148,74.817956,78.870986,94.765503,88.586840,43738,1 ETELÄINEN SUURPIIRI
6,Taka-Töölö,87.902482,95.685431,78.013245,90.008771,85.981219,15253,1 ETELÄINEN SUURPIIRI
7,Lauttasaari,70.974337,72.153629,51.920147,88.849234,76.159888,24736,1 ETELÄINEN SUURPIIRI
9,Reijola,100.060517,87.922642,110.102059,102.156851,100.630765,16767,2 LÄNTINEN SUURPIIRI
10,Munkkiniemi,85.070119,78.960570,82.743505,93.506284,90.378942,18844,2 LÄNTINEN SUURPIIRI
11,Haaga,104.312980,111.773187,101.344437,99.821315,102.880702,27612,2 LÄNTINEN SUURPIIRI
12,Pitäjänmäki,100.216510,92.222125,112.076408,96.350997,103.612896,18322,2 LÄNTINEN SUURPIIRI
13,Kaarela,104.938263,101.566463,110.505620,102.742707,109.150969,30500,2 LÄNTINEN SUURPIIRI


In [5]:
# Write morbidity dataframe to csv file in data folder
morb.to_csv('../data/Helsinki_morbidity_index.csv', index=False)